# Acquiring and processing LoVoCCS-relevant X-ray data

Here we use the Python module DAXA (Democratising Archival X-ray Astronomy) to identify X-ray observations relevant to the LoVoCCS sample, download them, and process them into a usable state.

DAXA is capable of searching the archives of a number of X-ray missions, though more will be added in the future. In this instance we search the following:
* XMM-Newton pointed observations
* Chandra observations
* ROSAT All-Sky Survey observations
* ROSAT pointed observations 
* NuSTAR pointed observations
* eROSITA calibration and performance verification observations
* eROSITA All Sky Survey 1DE
* Swift observations
* Suzaku observations
* ASCA observations

**Though XMM is the current main focus for data analysis, we acquire data for the other missions, including pre-processed images and exposure maps where possible** - in the future we will process and use the data from the other missions.

## Import Statements

In [1]:
import pandas as pd
from astropy.units import Quantity
from astropy.cosmology import LambdaCDM
import numpy as np
import os

import daxa
daxa.NUM_CORES = 70
daxa.OUTPUT = "/mnt/ufs18/rs-033/mc2/LoVoCCS/X-LoVoCCS-Data/data/"
daxa.config.OUTPUT = "/mnt/ufs18/rs-033/mc2/LoVoCCS/X-LoVoCCS-Data/data/"
daxa.mission.base.OUTPUT = "/mnt/ufs18/rs-033/mc2/LoVoCCS/X-LoVoCCS-Data/data/"
from daxa.mission import XMMPointed, Chandra, ROSATPointed, ROSATAllSky, NuSTARPointed, eROSITACalPV, Swift, \
    Suzaku, ASCA, eRASS1DE
from daxa.archive import Archive
from daxa.process.simple import full_process_xmm
from daxa.process.general import preprocessed_in_archive

print('DAXA - ', daxa.__version__)

DAXA -  v0.2.0+4.g4e7828f


## Useful functions

Here we set up any functions to be used in this notebook:

In [2]:
def rad_to_ang(rad, z, cosmology) -> Quantity:
    """
    Converts radius in length units to radius on sky in degrees.

    :param Quantity rad: Radius for conversion.
    :param Cosmology cosmology: An instance of an astropy cosmology, the default is a flat LambdaCDM concordance model.
    :param float z: The _redshift of the source.
    :return: The radius in degrees.
    :rtype: Quantity
    """
    d_a = cosmology.angular_diameter_distance(z)
    ang_rad = (rad.to("Mpc") / d_a).to('').value * (180 / np.pi)
    return Quantity(ang_rad, 'deg')

## Setting up cosmology

The original LoVoCCS paper uses a flat LambdaCDM cosmology, which was chosen to be consistent with simulations that the data will be compared to:

In [3]:
cosmo = LambdaCDM(71, 0.2648, 0.7352, Ob0=0.0448)

## Reading in the sample

We will be searching for data around the central positions of the LoVoCCS galaxy clusters, as defined by the MCXC catalogue. As such we need to read in our basic initial information about the clusters:

In [4]:
samp = pd.read_csv("../sample_files/lovoccs_southnorth.csv")
samp

,name,MCXC,LoVoCCSID,LoVoCCS_name,ra,dec,redshift,L500,M500,R500,alt_name,other_names,Notes,sub_samp,old_LoVoCCSID
0,MCXCJ1558.3+2713,J1558.3+2713,0,LoVoCCS-0,239.585833,27.226944,0.0894,10.676087,8.1491,1.3803,RXCJ1558.3+2713,A2142,L,north,0
1,MCXCJ1510.9+0543,J1510.9+0543,1,LoVoCCS-1,227.729167,5.720000,0.0766,8.726709,7.2708,1.3344,A2029,A2029,NaN,south,1
2,MCXCJ0258.9+1334,J0258.9+1334,2,LoVoCCS-2,44.739583,13.579444,0.0739,6.088643,5.8488,1.2421,RXCJ0258.9+1334,A401,L,south,2
3,MCXCJ1348.8+2635,J1348.8+2635,3,LoVoCCS-3,207.220833,26.595556,0.0622,5.478067,5.5280,1.2236,RXCJ1348.8+2635,A1795,NaN,north,3
4,MCXCJ0041.8-0918,J0041.8-0918,4,LoVoCCS-4,10.458750,-9.301944,0.0555,5.100085,5.3163,1.2103,RXCJ0041.8-0918,A85,"L,losStr",south,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,MCXCJ0448.2-2028,J0448.2-2028,142,LoVoCCS-142,72.050833,-20.469722,0.0720,1.004022,1.9513,0.8620,RXCJ0448.2-2028,A514,losStr,south,139
143,MCXCJ2323.8+1648,J2323.8+1648,143,LoVoCCS-143,350.972917,16.808889,0.0416,1.002026,1.9896,0.8760,A2589,A2589,NaN,north,140
144,MCXCJ1416.8-1158,J1416.8-1158,144,LoVoCCS-144,214.214583,-11.976111,0.0982,1.001648,1.9133,0.8491,RXCJ1416.8-1158,NaN,X,south,141
145,MCXCJ1459.0-0843,J1459.0-0843,145,LoVoCCS-145,224.764583,-8.725000,0.1043,1.001337,1.9047,0.8461,RXCJ1459.0-0843,NaN,NaN,south,142


## Searching for X-ray data

For many of these missions we wish to search for observations within some physically motivated radius, and in this case we decide to use a radius of 2$R_{500}$, where the $R_{500}$ in question is that measured by MCXC. As such we convert the Mpc value of 2$R_{500}$ to an angular search distance:

In [5]:
search_distances = rad_to_ang(Quantity(samp['R500'].values*2, 'Mpc'), samp['redshift'].values, cosmo)
search_distances

<Quantity [0.46490923, 0.51699791, 0.49728801, 0.57428836, 0.63172923,
           0.62477756, 0.30542619, 0.35266034, 0.35926306, 0.56708541,
           0.39874873, 0.61047514, 0.8369842 , 0.444773  , 0.45680494,
           0.38924364, 0.56131184, 0.29730888, 0.39034992, 0.29255113,
           0.33642153, 0.65873291, 0.3582728 , 0.39363645, 0.33842223,
           0.27518174, 0.32396228, 0.52586353, 0.40867011, 0.29431832,
           0.57176252, 0.36975457, 0.42808129, 0.29503709, 0.29296726,
           0.32363163, 0.26744286, 0.32233612, 0.31077669, 0.2985346 ,
           0.411502  , 0.35062178, 0.38987939, 0.35598946, 0.25516586,
           0.25732541, 0.32335499, 0.94418002, 0.6032685 , 0.24882547,
           0.3734829 , 0.30732523, 0.86301678, 0.36810844, 0.34900687,
           0.24818938, 0.50398473, 0.25618702, 0.32250687, 0.35833698,
           0.31861123, 0.26976724, 0.26848454, 0.24582944, 0.34492831,
           0.60630628, 0.30287874, 0.58249856, 0.29302049, 0.3171137 ,
      

Here we detail the meanings of the DAXA target category taxonomy codes; for reference when viewing the observation target category columns. Note that this may evolve with newer versions of DAXA:

In [6]:
XMMPointed.show_allowed_target_types()

╒═══════════════╤═══════════════════════════════════════════════╕
│ Target Type   │ Description                                   │
╞═══════════════╪═══════════════════════════════════════════════╡
│ AGN           │ Active Galaxies and Quasars                   │
├───────────────┼───────────────────────────────────────────────┤
│ BLZ           │ Blazars                                       │
├───────────────┼───────────────────────────────────────────────┤
│ CV            │ Cataclysmic Variables                         │
├───────────────┼───────────────────────────────────────────────┤
│ CAL           │ Calibration Observation (possibly of objects) │
├───────────────┼───────────────────────────────────────────────┤
│ EGS           │ Extragalactic Surveys                         │
├───────────────┼───────────────────────────────────────────────┤
│ GCL           │ Galaxy Clusters                               │
├───────────────┼───────────────────────────────────────────────┤
│ GS      

### XMM-Newton

Observations of LoVoCCS clusters by XMM-Newton are the initial focus of the X-LoVoCCS project, which is well suited to the study of local, large, galaxy clusters due to its medium sized field of view and excellent sensitivity. We select all X-ray instruments on XMM; all three EPIC detectors (PN, MOS1, and MOS2), as well as the grating spectrometer instruments (RGS1 and RGS2).

We initially need to declare an XMMPointed instance, which fetches the latest database of XMM observations, then we can filter all those observations to select only the ones we're interested in.

In [7]:
xm = XMMPointed(['PN', 'MOS1', 'MOS2', 'RGS1', 'RGS2'])
xm.all_obs_info

/mnt/ufs18/home-218/turne540/code/DAXA/daxa/mission/base.py:405: UserWarning: Some instrument names were converted to alternative forms expected by this module, the instrument names are now; PN, M1, M2, R1, R2
  self._chos_insts = self.check_inst_names(new_insts)
/mnt/ufs18/home-218/turne540/code/DAXA/daxa/mission/xmm.py:83: UserWarning: 140 of the 17976 observations located for this mission have been removed due to NaN RA or Dec values
  self._fetch_obs_info()


,ra,dec,ObsID,start,science_usable,duration,proprietary_end_date,revolution,proprietary_usable,end
0,64.925415,55.999440,0000110101,2001-08-19 07:05:23,True,0 days 09:08:33,2002-09-29,310,True,2001-08-19 16:13:56
1,263.674950,-32.581670,0001730201,2001-03-09 12:44:21,True,0 days 04:44:43,2002-05-25,229,True,2001-03-09 17:29:04
2,263.674950,-32.581670,0001730301,2001-03-09 17:30:16,True,0 days 02:36:02,2002-05-25,229,True,2001-03-09 20:06:18
3,263.674950,-32.581670,0001730401,2001-03-09 09:41:25,True,0 days 03:00:59,2002-05-25,229,True,2001-03-09 12:42:24
4,99.349995,6.135278,0001730501,2002-09-17 18:35:28,True,0 days 06:05:39,2004-12-31,508,True,2002-09-18 00:41:07
...,...,...,...,...,...,...,...,...,...,...
17831,103.801792,-1.481528,0935630101,2024-09-26 08:22:01,True,0 days 18:20:00,NaT,4542,False,2024-09-27 02:42:01
17832,261.475375,-35.932139,0944380801,2024-09-27 05:12:01,True,0 days 03:20:00,NaT,4542,False,2024-09-27 08:32:01
17833,255.705792,-48.789694,0953390101,2024-09-27 09:06:15,True,0 days 12:52:41,NaT,4542,False,2024-09-27 21:58:56
17834,234.891250,-83.592222,0942980101,2024-09-28 21:43:39,True,1 days 00:08:20,NaT,4543,False,2024-09-29 21:51:59


This is where we filter the observations, using the search distances we decided on earlier. This searches for observations with a central coordinate within a circle that has a radius equal to the search distance for the particular cluster.

We then show a preview of the filtered observation table:

In [8]:
xm.filter_on_positions(samp[['ra', 'dec']].values, search_distances)
xm.filtered_obs_info

,ra,dec,ObsID,start,science_usable,duration,proprietary_end_date,revolution,proprietary_usable,end
59,247.156500,39.552610,0008030201,2002-07-04 15:42:00,True,0 days 06:22:56,2003-09-12,470,True,2002-07-04 22:04:56
60,247.156500,39.552610,0008030301,2002-07-06 15:35:25,True,0 days 06:21:17,2003-09-12,471,True,2002-07-06 21:56:42
61,247.156500,39.552610,0008030601,2002-08-15 12:52:43,True,0 days 06:03:56,2003-09-12,491,True,2002-08-15 18:56:39
88,194.366700,-17.346280,0010420201,2001-01-08 15:58:43,True,0 days 06:20:06,2002-10-08,199,True,2001-01-08 22:18:49
89,194.366700,-17.346280,0010420701,2001-01-08 12:48:31,True,0 days 03:01:48,2002-10-08,199,True,2001-01-08 15:50:19
...,...,...,...,...,...,...,...,...,...,...
17371,351.045833,17.070000,0924120201,2023-12-26 23:46:20,True,0 days 17:20:00,2025-01-15,4404,False,2023-12-27 17:06:20
17378,350.937500,16.485556,0924120401,2024-01-01 23:09:21,True,0 days 17:30:00,2025-01-25,4407,False,2024-01-02 16:39:21
17424,44.787792,13.198250,0921670101,2024-02-04 16:25:49,True,0 days 21:55:00,2025-02-26,4424,False,2024-02-05 14:20:49
17804,72.142083,-20.562222,0943700101,2024-09-06 09:45:21,True,0 days 23:58:20,NaT,4532,False,2024-09-07 09:43:41


We note that some of the observations selected are still proprietary (as of the time of writing), so we will not yet be able to use them - they will be automatically filtered out by DAXA when it comes to the data-downloading stage, but will be useful in the future when this DAXA archive is updated:

In [9]:
xm.filtered_obs_info[xm.filtered_obs_info['proprietary_usable'] == False]

,ra,dec,ObsID,start,science_usable,duration,proprietary_end_date,revolution,proprietary_usable,end
16833,67.508500,-61.373750,0921880301,2023-06-13 17:40:47,True,1 days 12:53:20,2024-12-18,4306,False,2023-06-15 06:34:07
16834,67.508500,-61.373750,0921881001,2023-06-13 16:44:27,True,0 days 00:56:20,2024-12-18,4306,False,2023-06-13 17:40:47
16867,68.005125,-61.292611,0921880501,2023-06-25 19:42:49,True,0 days 21:40:00,2024-11-13,4312,False,2023-06-26 17:22:49
16868,68.005125,-61.292611,0921881301,2023-06-25 15:57:09,True,0 days 03:45:40,2024-11-13,4312,False,2023-06-25 19:42:49
16875,68.133417,-61.506250,0921880601,2023-06-27 19:34:45,True,0 days 22:13:20,2024-12-12,4313,False,2023-06-28 17:48:05
16876,68.133417,-61.506250,0921881401,2023-06-27 17:15:45,True,0 days 02:19:00,2024-12-12,4313,False,2023-06-27 19:34:45
17148,68.005125,-61.292611,0921880101,2023-10-17 12:09:28,True,1 days 08:45:00,2024-11-13,4369,False,2023-10-18 20:54:28
17149,68.005125,-61.292611,0921881701,2023-10-17 08:07:58,True,0 days 04:01:30,2024-11-13,4369,False,2023-10-17 12:09:28
17219,68.133417,-61.506250,0921880201,2023-11-22 09:23:00,True,1 days 10:03:20,2024-12-12,4387,False,2023-11-23 19:26:20
17220,68.133417,-61.506250,0921881901,2023-11-22 06:53:00,True,0 days 02:30:00,2024-12-12,4387,False,2023-11-22 09:23:00


### Chandra

Chandra observations of LoVoCCS clusters will allow us to explore the spatially resolved properties of the ICM with more confidence, thanks to its excellent spatial resolution. However the smaller field-of-view and lower sensitivity (when compared to XMM) mean that it will be a focus in a future phase of the project. 

We still wish to assess the number of observations that may be available with Chandra however, and we can use DAXA to download pre-processed data, with images that may be useful at this stage:

In [10]:
ch = Chandra()
ch.all_obs_info

,ra,dec,ObsID,science_usable,proprietary_usable,start,end,duration,proprietary_end_date,target_category,instrument,grating,data_mode
0,274.43140,-33.01883,6616,True,True,2006-02-24 04:33:41.000003,2007-09-21 00:26:11.000003,573 days 19:52:30,2007-02-28,GS,ACIS-S,HETG,CC_000A8
1,83.63292,22.01447,7587,True,True,2007-02-03 09:58:57.000000,2008-07-21 21:53:57.000000,534 days 11:55:00,2008-02-06,SNR,ACIS-S,HETG,TE_0077C
2,202.50000,47.20000,13814,True,True,2012-09-20 07:21:41.999999,2012-09-22 12:47:41.999999,2 days 05:26:00,2013-10-11,NGS,ACIS-S,NONE,TE_00958
3,266.41667,-29.00781,13842,True,True,2012-07-21 11:52:41.000002,2012-07-23 17:08:41.000002,2 days 05:16:00,2012-07-25,NGS,ACIS-S,HETG,TE_008D0
4,316.72458,38.74942,13651,True,True,2012-02-13 20:18:26.999997,2012-02-16 00:51:26.999997,2 days 04:33:00,2013-02-21,MISC,HRC-S,LETG,DEFAULT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24159,332.17000,45.74231,24645,True,True,2020-10-25 12:05:32.000001,2020-10-25 12:05:32.000001,0 days 00:00:00,2020-10-27,CAL,HRC-S,NONE,SCENTER
24160,54.19708,0.58997,1153,True,True,1999-09-05 04:29:00.999998,1999-09-05 04:29:00.999998,0 days 00:00:00,2002-10-04,CAL,HRC-S,NONE,SCENTER
24161,350.85750,58.81483,1409,True,True,1999-10-23 18:29:33.999999,1999-10-23 18:29:33.999999,0 days 00:00:00,1999-12-09,SNR,HRC-I,NONE,
24162,332.17000,45.74231,24644,True,True,2020-09-29 01:23:47.000003,2020-09-29 01:23:47.000003,0 days 00:00:00,2020-09-30,CAL,HRC-I,NONE,DEFAULT


This is the same procedure as with the search for XMM data, and we find that there are a significant number of observations available:

In [11]:
ch.filter_on_positions(samp[['ra', 'dec']].values, search_distances)
ch.filtered_obs_info

/mnt/ufs18/home-218/turne540/code/DAXA/daxa/mission/base.py:1396: UserWarning: Chandra FoV are difficult to define, as they can be strongly dependant on observation mode; as such take these as very approximate.
  fov = self.fov
/mnt/ufs18/home-218/turne540/code/DAXA/daxa/mission/base.py:108: UserWarning: The mission has FoVs defined for ACIS-I, ACIS-S, HRC-I, HRC-S, but only one search_radius has been supplied. You may wish to pass a dictionary of search radii.
  any_ret = change_func(*args, **kwargs)


,ra,dec,ObsID,science_usable,proprietary_usable,start,end,duration,proprietary_end_date,target_category,instrument,grating,data_mode
13,15.35027,-21.81757,13442,True,True,2011-08-23 07:02:35.999998,2011-08-25 08:46:15.999998,2 days 01:43:40,2011-08-30,GCL,ACIS-I,NONE,TE_004D8
66,193.22708,-29.45500,4198,True,True,2003-03-20 10:23:35.000002,2003-03-22 08:23:35.000002,1 days 22:00:00,2003-03-26,GCL,ACIS-I,NONE,TE_0048A
135,15.99304,-21.71045,13448,True,True,2011-09-13 09:56:39.000002,2011-09-15 03:04:09.000002,1 days 17:07:30,2011-09-16,GCL,ACIS-I,NONE,TE_004D8
147,332.74768,-12.42057,23368,True,True,2021-07-22 02:38:25.999996,2021-07-23 18:40:55.999996,1 days 16:02:30,2022-07-26,MISC,HRC-I,NONE,OBS20743
149,15.32166,-22.08948,13452,True,True,2011-09-24 12:29:40.999998,2011-09-26 04:28:50.999998,1 days 15:59:10,2011-09-27,GCL,ACIS-I,NONE,TE_004D8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20522,247.25542,40.13322,4140,True,True,2003-07-12 08:26:28.000003,2003-07-12 09:23:38.000003,0 days 00:57:10,2004-07-16,AGN,ACIS-S,NONE,TE_003C4
20627,176.15333,67.40592,5675,True,True,2005-03-25 21:37:35.999999,2005-03-25 22:30:35.999999,0 days 00:53:00,2006-03-28,AGN,ACIS-S,NONE,TE_006A0
20911,256.04479,78.63100,1521,True,True,2000-02-27 01:49:23.000005,2000-02-27 02:39:03.000005,0 days 00:49:40,2002-02-14,GCL,ACIS-S,NONE,TE_002DE
21917,227.92208,5.30264,4047,True,True,2003-05-18 11:19:00.999995,2003-05-18 11:52:40.999995,0 days 00:33:40,2004-06-04,AGN,ACIS-S,NONE,TE_002A2


### ROSAT All-Sky

All of our clusters will be covered by the ROSAT All-Sky Survey, and data from this mission will be helpful in defining the background emission when analysing our clusters with XMM. It is also useful to have a data source that provides uniform(-ish) coverage of our sample, it provides a base for our work to build on:

In [12]:
ra = ROSATAllSky()
ra.all_obs_info

,ra,dec,ObsID,science_usable,start,end,duration,target_category
0,263.57088,67.500,RS930521N00,True,1990-07-11,1991-08-13,0 days 11:19:06,ASK
1,276.42533,67.500,RS930522N00,True,1990-07-11,1991-08-13,0 days 11:18:11,ASK
2,96.42533,-67.500,RS932908N00,True,1990-07-11,1991-08-13,0 days 05:47:48,ASK
3,83.57088,-67.500,RS932907N00,True,1990-07-11,1991-08-13,0 days 05:47:39,ASK
4,267.27100,61.875,RS930625N00,True,1990-07-11,1991-08-13,0 days 03:44:35,ASK
...,...,...,...,...,...,...,...,...
1373,289.08783,-61.875,RS932827N00,True,1990-09-10,1990-10-08,0 days 00:02:44,ASK
1374,350.17942,-33.750,RS932354N00,True,1990-11-09,1990-12-02,0 days 00:02:42,ASK
1375,278.17942,-61.875,RS932826N00,True,1990-09-04,1990-09-30,0 days 00:02:42,ASK
1376,273.75000,-45.000,RS932537N00,True,1990-09-07,1990-09-23,0 days 00:02:42,ASK


The RASS survey data is divided into 6x6 degree chunks of sky, and as such our previously defined search distances are not a good choice here - it is entirely possible that the centre of the relevant RASS chunk will be further away than the search distance, even though the cluster does lie in that chunk. As such we use the default search distance (which is based upon the known size of the RASS chunks).

We find that 144 RASS 'observations' are selected, which may correspond to one per cluster:

In [13]:
ra.filter_on_positions(samp[['ra', 'dec']].values)
ra.filtered_obs_info

,ra,dec,ObsID,science_usable,start,end,duration,target_category
9,256.36258,61.875,RS930624N00,True,1990-07-11,1991-08-13,0 days 01:17:00,ASK
21,258.75000,78.750,RS930312N00,True,1990-07-30,1991-08-13,0 days 00:45:33,ASK
25,70.90838,-61.875,RS932807N00,True,1990-07-11,1991-08-13,0 days 00:41:47,ASK
27,96.92104,-56.250,RS932711N00,True,1990-08-16,1991-02-18,0 days 00:40:29,ASK
41,244.61279,56.250,RS930727N00,True,1990-07-11,1991-08-13,0 days 00:30:27,ASK
...,...,...,...,...,...,...,...,...
1316,234.00000,-84.375,RS933207N00,True,1990-08-15,1991-02-18,0 days 00:05:11,ASK
1317,334.68750,-5.625,RS931860N00,True,1990-11-08,1990-11-28,0 days 00:05:10,ASK
1350,349.61279,-39.375,RS932451N00,True,1990-11-04,1990-11-29,0 days 00:03:50,ASK
1365,345.24583,-22.500,RS932159N00,True,1990-11-11,1990-12-02,0 days 00:03:10,ASK


### ROSAT Pointed

We also wish to search for pointed ROSAT observations of our clusters, as they may be useful. Pointed PSPC observations could be spectrally analysed for instance, though if the observations were off axis the PSF effects can become quite extreme. Pointed HRI observations have very good spatial resolution, so could be used for the assessment of ICM structure, but very poor spectral resolution, so would not be useful for spectral fitting:

In [14]:
rp = ROSATPointed()
rp.all_obs_info

,ra,dec,ObsID,science_usable,start,end,duration,instrument,with_filter,target_category,target_name,proc_rev,fits_type
0,163.1800,57.4800,RH701867A01,True,1995-04-15 23:24:16.000001,1995-05-11 14:24:47.000001,2 days 13:29:32,HRI,N,AGN,LOCKMAN HOLE,2,RFITS V4.
1,203.6500,37.9100,RH900717N00,True,1997-06-04 16:13:00.999998,1997-07-13 22:26:45.000004,2 days 07:58:33,HRI,N,MISC,DEEP SURVEY,2,RDF 4_0
2,163.1800,57.4800,RH701867A04,True,1997-04-15 21:51:16.999998,1997-04-28 16:37:45.999998,2 days 06:33:41,HRI,N,AGN,LOCKMAN HOLE,2,RFITS V3.
3,350.8700,58.8100,RH500444N00,True,1995-12-23 22:18:36.999999,1996-02-01 10:17:53.999998,2 days 02:07:27,HRI,N,SNR,CAS A,2,RDF 3_4
4,163.1800,57.4800,RH701867A02,True,1996-05-01 02:09:33.000002,1996-05-29 15:31:24.999997,2 days 01:05:24,HRI,N,AGN,LOCKMAN HOLE,2,RFITS V4.
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11426,84.2900,-80.4700,RP999998A02,False,1991-10-20 04:07:51.999998,1991-11-01 22:43:43.000003,0 days 00:00:00,PSPCB,N,EGE,Idle Point,2,RDF 3_4
11427,93.1800,-81.8300,RH150094N00,False,1990-07-29 22:13:42.999997,1990-07-29 22:56:58.999998,0 days 00:00:00,HRI,N,STR,Calibration Source,2,RFITS V4.
11428,218.1540,-44.2031,RH001034N00,False,1997-08-28 00:29:42.000000,1997-08-28 01:09:58.999997,0 days 00:00:00,HRI,N,MISC,,2,RDF 4_2
11429,258.1383,-23.3850,RH800067M01,False,1991-03-19 20:33:48.999997,1991-03-19 20:35:59.999997,0 days 00:00:00,HRI,N,GCL,OPHIUCHUS CLUSTER,2,RFITS V3.


We again use the default search distances - this is due to HRI and PSPC having _very_ different field of views, so using a single search distance is not really valid. We also exclude one selected observation due to an incompatibility with DAXA that only appears when we try to download the data:

In [15]:
rp.filter_on_positions(samp[['ra', 'dec']].values)

# Due to a different processing version to the rest of the data (I assume), RP700068 cannot currently be downloaded
#  by DAXA, so we're manually excluding it
rp.filter_array[np.where(rp.all_obs_info['ObsID'] == 'RP700068')[0]] = False

rp.filtered_obs_info

,ra,dec,ObsID,science_usable,start,end,duration,instrument,with_filter,target_category,target_name,proc_rev,fits_type
178,348.4900,-42.7300,RH800448N00,True,1994-11-05 12:42:18.000000,1994-11-27 04:14:48.999998,0 days 13:23:40,HRI,N,GCL,SERSIC 159-03,2,RDF 3_8
245,255.8400,78.6400,RH800676N00,True,1995-03-02 12:38:52.999998,1995-03-06 03:04:16.000003,0 days 12:06:07,HRI,N,GCL,A2256,2,RDF 3_4
248,303.1300,-56.8200,RH800822N00,True,1996-04-06 16:10:35.000000,1996-04-11 13:08:29.000002,0 days 11:59:23,HRI,N,GCL,A3667,2,RDF 3_4
283,247.1600,39.5500,RP800644N00,True,1993-07-25 12:07:22.999999,1993-07-28 21:49:24.000004,0 days 11:23:19,PSPCB,N,GCL,A2199,2,RDF 3_4
335,137.2100,-9.6500,RH800768A01,True,1996-04-26 07:31:21.000000,1996-04-28 15:47:16.000002,0 days 10:22:26,HRI,N,GCL,A754,2,RDF 3_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10019,229.1800,7.0200,RH800223A01,True,1994-07-31 03:09:02.999998,1994-07-31 21:01:26.000003,0 days 00:13:36,HRI,N,GCL,A2052,2,RDF 3_6
10112,174.1300,67.6200,RP900521A01,True,1994-05-04 09:12:06.000002,1994-05-04 09:26:54.999997,0 days 00:12:33,PSPCB,N,EGE,HS 1133+6753,2,RDF 3_4
10509,230.7700,8.6099,RP800184A00,True,1992-02-01 10:04:44.000000,1992-02-01 10:08:44.000002,0 days 00:06:31,PSPCB,N,GCL,A 2063,2,RDF 3_4
10545,255.9000,78.7200,RP100145N00,True,1990-06-20 00:28:57.000003,1990-06-21 22:48:14.999999,0 days 00:05:52,PSPCC,N,STR,XRT/PSPC SPEC/FLUX B,2,RDF 3_3


We can examine the number of pointed observations for each instrument:

In [16]:
rp.filtered_obs_info['instrument'].value_counts()

instrument
PSPCB    126
HRI       93
PSPCC      5
Name: count, dtype: int64

### NuSTAR Pointed

NuSTAR is not a typical choice for the study of galaxy clusters, but it has been used for this purpose in the past. It is a very capable telescope which may provide extra coverage of our galaxy clusters, and it is also unique in the current selection of missions in that it can observe at very high energies (relative to the typical X-ray range) - this may be useful in future LoVoCCS work, or it may not, but it will be interesting to explore.

This mission does not have a large field of view, and has a limited number of requested pointings at galaxy clusters, so we do not expect very many observations to be selected:

In [17]:
nu = NuSTARPointed()
nu.all_obs_info

,ra,dec,ObsID,science_usable,proprietary_usable,start,end,duration,proprietary_end_date,target_category,exposure_a,exposure_b,ontime_a,ontime_b,nupsdout,issue_flag
0,83.8281,-69.2466,40001014013,True,True,2013-06-29 01:16:10.560,2013-07-05 08:51:04.320,6 days 07:34:53.760000,2015-09-17,MISC,5 days 11:20:42,5 days 11:09:58,5 days 20:32:48,5 days 20:33:46,2119,1
1,83.7759,-69.2678,40001014016,True,True,2014-04-22 21:06:02.880,2014-04-29 09:31:06.240,6 days 12:25:03.360000,2015-09-17,MISC,4 days 23:56:47,4 days 23:35:17,5 days 08:14:46,5 days 08:14:09,0,1
2,83.8965,-69.2478,40001014023,True,True,2014-08-01 23:46:10.560,2014-08-08 02:41:08.160,6 days 02:54:57.600000,2015-09-17,MISC,4 days 22:46:56,4 days 22:35:32,5 days 07:01:07,5 days 07:02:55,0,0
3,340.6530,29.6985,60401031004,True,True,2018-11-28 22:21:12.960,2018-12-08 17:16:13.440,9 days 18:55:00.480000,2019-12-25,AGN,4 days 17:35:58,4 days 17:01:43,5 days 02:48:25,5 days 02:49:43,0,0
4,344.4048,-36.9766,60901012002,True,True,2023-11-02 06:06:11.520,2023-11-10 11:41:08.160,8 days 05:34:56.640000,2024-05-20,AGN,4 days 11:03:16,4 days 10:06:55,4 days 19:02:15,4 days 19:02:36,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5537,0.0000,0.0000,20511422001,True,True,2019-04-26 03:52:07.680,2019-04-26 03:54:43.200,0 days 00:02:35.520000,2019-05-08,SOL,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0,1
5538,0.0000,0.0000,20511423001,True,True,2019-04-26 03:54:43.200,2019-04-26 03:57:10.080,0 days 00:02:26.880000,2019-05-08,SOL,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0,1
5539,0.0000,0.0000,20511424001,True,True,2019-04-26 03:57:10.080,2019-04-26 03:59:36.960,0 days 00:02:26.880000,2019-05-08,SOL,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0,1
5540,0.0000,0.0000,20511425001,True,True,2019-04-26 03:59:36.960,2019-04-26 04:06:05.760,0 days 00:06:28.800000,2019-05-08,SOL,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0,1


We search with the search distances defined earlier again, because they are likely to be larger than the telescope field of view:

In [18]:
nu.filter_on_positions(samp[['ra', 'dec']].values, search_distances)
nu.filtered_obs_info

,ra,dec,ObsID,science_usable,proprietary_usable,start,end,duration,proprietary_end_date,target_category,exposure_a,exposure_b,ontime_a,ontime_b,nupsdout,issue_flag
37,303.1217,-56.8508,70901001002,True,False,2024-06-17 22:21:12.960,2024-06-22 16:21:12.960,4 days 18:00:00,2025-01-01,GCL,2 days 09:48:34,2 days 09:14:56,2 days 14:35:19,2 days 14:36:38,0,1
48,349.0706,-42.6139,50501001002,True,True,2019-10-10 16:16:10.560,2019-10-14 18:26:12.480,4 days 02:10:01.920000,2020-10-30,NGS,2 days 05:02:44,2 days 04:42:39,2 days 08:46:54,2 days 08:47:33,0,0
158,96.5925,-54.3964,70601003002,True,True,2020-09-09 17:46:10.560,2020-09-12 07:26:06.720,2 days 13:39:56.160000,2021-09-21,GCL,1 days 11:52:57,1 days 11:31:53,1 days 14:21:07,1 days 14:21:25,0,0
175,137.4240,-9.6896,70201001002,True,True,2017-01-01 05:56:06.720,2017-01-03 23:21:07.200,2 days 17:25:00.480000,2018-01-10,GCL,1 days 10:23:26,1 days 10:18:25,1 days 12:49:04,1 days 12:49:10,0,0
177,258.1640,64.0542,70901002002,True,True,2023-10-11 16:36:11.520,2023-10-14 03:51:07.200,2 days 11:14:55.680000,2024-04-23,GCL,1 days 10:12:46,1 days 09:53:26,1 days 12:49:57,1 days 12:50:44,0,0
308,255.9702,78.6158,70001053002,True,True,2013-03-09 20:21:07.200,2013-03-11 18:41:11.040,1 days 22:20:03.840000,2014-02-06,GCL,1 days 04:24:27,1 days 04:23:09,1 days 06:25:03,1 days 06:25:49,0,0
320,227.7529,5.7465,70660001002,True,True,2021-02-17 20:51:12.960,2021-02-20 09:36:08.640,2 days 12:44:55.680000,2022-03-01,GCL,1 days 04:15:49,1 days 03:59:56,1 days 06:27:12,1 days 06:27:39,0,0
337,207.1966,26.5877,70660003002,True,True,2020-07-15 14:26:09.600,2020-07-17 22:46:07.680,2 days 08:19:58.080000,2021-07-27,GCL,1 days 04:00:30,1 days 03:46:18,1 days 06:03:31,1 days 06:04:06,0,0
386,67.8080,-61.4553,70801001002,True,True,2022-11-17 00:31:06.240,2022-11-19 01:56:12.480,2 days 01:25:06.240000,2023-06-05,GCL,1 days 02:28:26,1 days 02:13:14,1 days 04:18:25,1 days 04:18:34,0,0
465,247.1378,39.5406,70660004002,True,True,2020-09-18 07:01:12.000,2020-09-20 06:56:09.600,1 days 23:54:57.600000,2021-09-28,GCL,0 days 23:46:43,0 days 23:33:57,1 days 01:31:43,1 days 01:31:55,0,0


We also check to see how many of the observations are still proprietary, and find that there are several which have been requested to point at galaxy clusters (see the target category):

In [19]:
nu.filtered_obs_info[nu.filtered_obs_info['proprietary_usable'] == False]

,ra,dec,ObsID,science_usable,proprietary_usable,start,end,duration,proprietary_end_date,target_category,exposure_a,exposure_b,ontime_a,ontime_b,nupsdout,issue_flag
37,303.1217,-56.8508,70901001002,True,False,2024-06-17 22:21:12.960,2024-06-22 16:21:12.960,4 days 18:00:00,2025-01-01,GCL,2 days 09:48:34,2 days 09:14:56,2 days 14:35:19,2 days 14:36:38,0,1


### eROSITA Calibration and Performance Verification 

The eROSITA telescope is an excellent choice for the exploration of the ICM of bright, local, clusters at large scales. As of the time of writing there have been no data releases for the eROSITA All-Sky Survey (though they will hopefully be supported by DAXA soon after release), so we can only search the performance verification data, which is a mixture of pointed and slew observations.

In [20]:
er = eROSITACalPV()
er.all_obs_info

,ra,dec,ObsID,science_usable,start,end,duration,Field_Name,Field_Type,active_insts
0,129.550000,1.500000,300007,True,2019-11-03 02:42:50,2019-11-04 03:36:37,89627.0,EFEDS,SURVEY,"TM1,TM2,TM3,TM4,TM5,TM6,TM7"
1,133.860000,1.500000,300008,True,2019-11-04 03:49:16,2019-11-05 05:16:39,91643.0,EFEDS,SURVEY,"TM1,TM2,TM3,TM4,TM5,TM6,TM7"
2,138.140000,1.500000,300009,True,2019-11-05 05:29:18,2019-11-06 06:40:06,90648.0,EFEDS,SURVEY,"TM1,TM2,TM3,TM4,TM5,TM6,TM7"
3,142.450000,1.500000,300010,True,2019-11-06 07:24:46,2019-11-07 08:20:08,89722.0,EFEDS,SURVEY,"TM1,TM2,TM3,TM4,TM5,TM6,TM7"
4,130.331300,-78.963400,300004,True,2019-11-16 23:14:40,2019-11-18 18:17:12,154952.0,ETA_CHA,SURVEY,"TM1,TM2,TM3,TM4,TM5,TM6,TM7"
...,...,...,...,...,...,...,...,...,...,...
165,284.146250,-37.909167,700008,True,2019-10-24 11:11:19,2019-10-25 08:55:52,78273.0,1RXS_J185635_375433,EXTRAGALACTIC_FIELDS,"TM1,TM2,TM3,TM4,TM5,TM6,TM7"
166,281.540771,79.873726,900060,True,2019-09-24 15:27:06,2019-09-24 21:30:32,21806.0,3C390_3,EXTRAGALACTIC_FIELDS,TM6
167,281.500275,79.885376,900068,True,2019-09-28 15:49:51,2019-09-28 21:30:32,20441.0,3C390_3,EXTRAGALACTIC_FIELDS,TM6
168,281.489410,79.888214,900069,True,2019-09-29 15:23:24,2019-09-29 21:30:37,22033.0,3C390_3,EXTRAGALACTIC_FIELDS,TM6


We search the available eROSITA data and find that two of our LoVoCCS clusters have been observed during the commissioning phase - these were pointed observations, with a good exposure time. They have already been well analysed but will no doubt be useful to us in combination with the LoVoCCS weak-lensing data:

In [21]:
er.filter_on_positions(samp[['ra', 'dec']].values, search_distances)
er.filtered_obs_info

/mnt/ufs18/home-218/turne540/code/DAXA/daxa/mission/base.py:1396: UserWarning: A field-of-view cannot be easily defined for eROSITACalPV and this number is the approximate half-length of an eFEDS section, the worst case separation - this is unnecessarily large for pointed observations, and you should make your own judgement on a search distance.
  fov = self.fov


,ra,dec,ObsID,science_usable,start,end,duration,Field_Name,Field_Type,active_insts
148,55.72822,-53.63013,700177,True,2019-11-21 10:22:18,2019-11-22 08:36:11,80033.0,A3158,EXTRAGALACTIC_FIELDS,"TM1,TM2,TM3,TM4,TM5,TM6,TM7"
149,67.84096,-61.44033,700154,True,2019-11-11 02:09:28,2019-11-11 22:46:56,74248.0,A3266,EXTRAGALACTIC_FIELDS,"TM1,TM2,TM3,TM4,TM5,TM6,TM7"
161,96.70600,-54.07200,300014,True,2019-10-07 14:38:03,2019-10-08 01:49:29,40286.0,A3391_A3395,EXTRAGALACTIC_FIELDS,"TM5,TM6,TM7"
162,96.70600,-54.07200,300005,True,2019-10-08 15:54:34,2019-10-09 08:56:47,61333.0,A3391_A3395,EXTRAGALACTIC_FIELDS,"TM5,TM6,TM7"
163,96.70600,-54.07200,300006,True,2019-10-09 09:03:26,2019-10-10 01:52:33,60547.0,A3391_A3395,EXTRAGALACTIC_FIELDS,"TM5,TM6,TM7"
164,96.70600,-54.07200,300016,True,2019-10-17 23:59:56,2019-10-18 16:48:18,60502.0,A3391_A3395,EXTRAGALACTIC_FIELDS,"TM1,TM2,TM3,TM4,TM5,TM6,TM7"


### eROSITA All-Sky Survey 1DE

The first data release from the eROSITA All-Sky Survey (eRASS) is now public, and covers much of the southern sky. We search for sky tiles relevant to our sample, as it may provide some information for those clusters with no modern existing X-ray data:

In [22]:
ea = eRASS1DE()
ea.all_obs_info

,ra,dec,ObsID,science_usable,start,end,duration,ra_min,ra_max,dec_min,dec_max,neigh_obs
0,116.703297,42.008289,117048,True,2020-04-15 10:13:39,2020-04-19 02:13:48,3 days 16:00:09,114.725275,118.681319,40.5,43.5,"114045,118045,122045,113048,121048,112051,1160..."
1,120.659341,42.008289,121048,True,2020-04-17 10:14:32,2020-04-21 10:13:37,3 days 23:59:05,118.681319,122.637363,40.5,43.5,"118045,122045,117048,125048,116051,119051,1230..."
2,124.615385,42.008289,125048,True,2020-04-19 14:14:56,2020-04-23 14:14:20,3 days 23:59:24,122.637363,126.593407,40.5,43.5,"122045,126045,121048,129048,123051,127051,0000..."
3,128.571429,42.008289,129048,True,2020-04-21 18:15:19,2020-04-25 18:14:56,3 days 23:59:37,126.593407,130.549451,40.5,43.5,"126045,130045,125048,133048,127051,131051,0000..."
4,132.527473,42.008289,133048,True,2020-04-23 22:15:47,2020-04-28 02:15:10,4 days 03:59:23,130.549451,134.505495,40.5,43.5,"130045,134045,129048,136048,131051,135051,0000..."
...,...,...,...,...,...,...,...,...,...,...,...,...
2442,220.000000,-87.045181,220177,True,2020-03-19 20:42:36,2020-04-03 04:43:25,14 days 08:00:49,200.000000,240.000000,-88.5,-85.5,"001180,191174,214174,236174,259174,180177,2601..."
2443,260.000000,-87.045181,260177,True,2020-03-25 16:41:00,2020-04-05 12:43:45,10 days 20:02:45,240.000000,280.000000,-88.5,-85.5,"001180,236174,259174,281174,304174,220177,3001..."
2444,300.000000,-87.045181,300177,True,2020-03-28 16:42:43,2020-04-09 04:42:26,11 days 11:59:43,280.000000,320.000000,-88.5,-85.5,"001180,259174,281174,304174,326174,260177,3401..."
2445,340.000000,-87.045181,340177,True,2020-03-31 20:42:37,2020-04-12 00:44:26,11 days 04:01:49,320.000000,360.000000,-88.5,-85.5,"001180,304174,326174,349174,011174,300177,0201..."


In [23]:
ea.filter_on_positions(samp[['ra', 'dec']].values)
ea.filtered_obs_info

,ra,dec,ObsID,science_usable,start,end,duration,ra_min,ra_max,dec_min,dec_max,neigh_obs
13,111.789474,39.007499,112051,True,2020-04-13 02:11:25,2020-04-16 18:10:51,3 days 15:59:26,109.894737,113.684211,37.5,40.5,"109048,113048,117048,108051,116051,107054,1110..."
14,115.578947,39.007499,116051,True,2020-04-15 06:11:31,2020-04-18 22:11:18,3 days 15:59:47,113.684211,117.473684,37.5,40.5,"113048,117048,121048,112051,119051,111054,1150..."
28,168.631579,39.007499,169051,True,2020-05-16 06:20:43,2020-05-21 14:20:34,5 days 07:59:51,166.736842,170.526316,37.5,40.5,"164048,168048,172048,165051,172051,165054,1690..."
135,151.666667,27.004838,152063,True,2020-05-09 14:09:47,2020-05-14 02:09:30,4 days 11:59:43,150.000000,153.333333,25.5,28.5,"149060,153060,156060,148063,155063,148066,1510..."
172,150.810811,24.004214,151066,True,2020-05-09 22:07:43,2020-05-14 10:07:17,4 days 11:59:34,149.189189,152.432432,22.5,25.5,"148063,152063,155063,148066,154066,148069,1510..."
...,...,...,...,...,...,...,...,...,...,...,...,...
2399,8.181818,-81.037609,008171,True,2020-04-13 20:42:48,2020-04-23 00:45:24,9 days 04:02:36,0.000000,16.363636,-82.5,-79.5,"006168,019168,354168,025171,352171,011174,0341..."
2400,24.545455,-81.037609,025171,True,2020-04-13 12:46:21,2020-04-23 04:47:26,9 days 16:01:05,16.363636,32.727273,-82.5,-79.5,"006168,019168,032168,008171,041171,011174,0341..."
2402,57.272727,-81.037609,057171,True,2020-04-06 16:48:33,2020-04-21 16:49:05,15 days 00:00:32,49.090909,65.454545,-82.5,-79.5,"045168,058168,071168,041171,074171,034174,0561..."
2423,56.250000,-84.042970,056174,True,2020-04-02 16:46:59,2020-04-16 04:47:47,13 days 12:00:48,45.000000,67.500000,-85.5,-82.5,"041171,057171,074171,034174,079174,020177,0601..."


### Swift

We search the Swift XRT and UVOT archives for relevant observations; BAT may be included later:

In [24]:
sw = Swift(['XRT', 'UVOT'])
sw.all_obs_info

/mnt/ufs18/home-218/turne540/code/DAXA/daxa/mission/swift.py:122: UserWarning: 623 of the 365659 observations located for Swift have been removed due to all instrument exposures being zero.
  self._fetch_obs_info()
/mnt/ufs18/home-218/turne540/code/DAXA/daxa/mission/swift.py:122: UserWarning: 5356 of the 365659 observations located for Swift have been removed due to all chosen instrument (XRT, UVOT) exposures being zero.
  self._fetch_obs_info()


,ra,dec,ObsID,science_usable,start,end,duration,target_category,xrt_exposure,bat_exposure,uvot_exposure,roll_angle
0,328.70739,16.89364,00173780007,True,2005-12-25 00:42:01.999999,2005-12-28 00:00:23.000000,2 days 23:18:21.000001,MISC,1 days 02:44:09.014000,1 days 00:04:10,1 days 02:40:01.030000,224.66184
1,39.95079,-25.21897,00131560002,True,2005-06-03 23:59:00.999998,2005-06-06 22:47:30.000002,2 days 22:48:29.000004,MISC,0 days 22:36:12.946000,0 days 22:52:03,0 days 17:46:44.297000,37.50837
2,228.46115,30.87659,00164268014,True,2005-12-03 01:00:00.999996,2005-12-06 00:07:43.000000,2 days 23:07:42.000004,MISC,0 days 19:38:55.384000,0 days 19:59:15,0 days 00:00:00,146.36382
3,350.64958,5.74990,00148833003,True,2005-08-06 00:26:11.999996,2005-08-09 00:50:56.000003,3 days 00:24:44.000007,MISC,0 days 19:04:07.125000,0 days 19:23:36,0 days 18:21:43.023000,49.28614
4,75.27823,45.28513,00321174011,True,2008-08-31 02:25:00.999998,2008-09-03 00:34:06.999998,2 days 22:09:06,MISC,0 days 18:49:28.970000,0 days 21:08:01.650000,0 days 18:24:15.039000,81.38752
...,...,...,...,...,...,...,...,...,...,...,...,...
359675,95.60471,13.82661,03104644003,True,2022-04-12 15:55:39.999999,2022-04-12 16:50:29.999999,0 days 00:54:50,MISC,0 days 00:00:00,0 days 00:01:34,0 days 00:01:25.472000,280.01671
359676,218.19468,46.03669,00091955001,True,2014-06-04 06:00:59.000002,2014-06-04 06:57:26.999997,0 days 00:56:27.999995,MISC,0 days 00:00:00,0 days 00:03:38,0 days 00:03:08.885000,330.63197
359677,171.08870,14.20105,00055250009,True,2005-02-22 00:27:01.000002,2005-02-22 03:26:02.000000,0 days 02:59:00.999998,MISC,0 days 00:00:00,0 days 00:15:00,0 days 00:00:07.922000,40.89852
359678,7.54879,-73.15102,00048734117,True,2019-04-08 23:58:12.000000,2019-04-09 00:52:10.999998,0 days 00:53:58.999998,MISC,0 days 00:00:00,0 days 00:00:50,0 days 00:00:42.113000,5.93505


A significant number of Swift observations are near to members of our sample - typically Swift observations are short, but in some cases there may be X-ray data of a useful quality - the UVOT observations may be much more useful however:

In [25]:
sw.filter_on_positions(samp[['ra', 'dec']].values, search_distances)
sw.filtered_obs_info

/mnt/ufs18/home-218/turne540/code/DAXA/daxa/mission/base.py:108: UserWarning: The mission has FoVs defined for XRT, BAT, UVOT, but only one search_radius has been supplied. You may wish to pass a dictionary of search radii.
  any_ret = change_func(*args, **kwargs)


,ra,dec,ObsID,science_usable,start,end,duration,target_category,xrt_exposure,bat_exposure,uvot_exposure,roll_angle
165,68.97940,-12.62514,00348650001,True,2009-04-07 12:02:00.000001,2009-04-08 14:25:07.000003,1 days 02:23:07.000002,MISC,0 days 10:03:51.252000,0 days 10:13:13,0 days 10:02:12.132000,287.68290
422,227.70635,5.72669,00035187004,True,2005-09-21 04:45:01.999999,2005-09-22 23:19:12.000003,1 days 18:34:10.000004,MISC,0 days 07:05:58.145000,0 days 07:17:34,0 days 06:18:16.414000,256.59354
614,255.49014,34.24709,00048079002,True,2011-11-27 08:51:59.000003,2011-11-28 19:28:51.000001,1 days 10:36:51.999998,MISC,0 days 05:58:15.495000,0 days 06:00:26,0 days 05:57:54.793000,186.10598
959,255.49962,34.23314,00048079012,True,2011-12-11 00:36:59.999996,2011-12-12 20:48:04.000003,1 days 20:11:04.000007,MISC,0 days 05:11:38.945000,0 days 05:14:16,0 days 05:10:32.979000,171.41364
1033,194.84279,-3.97845,00049401065,True,2013-11-24 12:36:59.000003,2013-11-26 00:46:25.000000,1 days 12:09:25.999997,MISC,0 days 05:03:59.064000,0 days 05:06:04,0 days 05:02:47.886000,124.65797
...,...,...,...,...,...,...,...,...,...,...,...,...
358069,331.35790,-5.65929,07033402001,True,2023-05-18 15:05:29.999996,2023-05-18 15:59:32.000001,0 days 00:54:02.000005,MISC,0 days 00:00:02.497000,0 days 00:00:07,0 days 00:00:00.496000,69.91745
358085,350.90254,16.65245,07033007001,True,2023-05-16 20:16:03.000003,2023-05-16 21:09:47.000001,0 days 00:53:43.999998,MISC,0 days 00:00:02.497000,0 days 00:00:08,0 days 00:00:00.861000,77.25810
358269,137.05172,-10.00967,00074330047,True,2021-01-31 00:40:34.000000,2021-01-31 01:14:13.999998,0 days 00:33:39.999998,MISC,0 days 00:00:01.975000,0 days 00:24:10,0 days 00:00:00,177.30688
358383,137.07084,-9.99539,00074330020,True,2011-09-29 10:20:00.000004,2011-09-29 12:06:16.999998,0 days 01:46:16.999994,MISC,0 days 00:00:01.974000,0 days 00:05:09,0 days 00:00:00,83.96647


### Suzaku

We search the Suzaku archive for relevant observations - it may lack the spatial resolution of XMM (and especially Chandra), but could still be useful in cases where other observations are lacking - the CCD energy resolution is also quite high:

In [26]:
sz = Suzaku()
sz.all_obs_info

/mnt/ufs18/home-218/turne540/code/DAXA/daxa/mission/suzaku.py:109: UserWarning: 14 of the 3055 observations located for Suzaku have been removed due to all instrument exposures being zero.
  self._fetch_obs_info()
/mnt/ufs18/home-218/turne540/code/DAXA/daxa/mission/suzaku.py:297: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['EGS' 'GS' 'EGS' ... 'GCL' 'GCL' 'EGS']' has dtype incompatible with int16, please explicitly cast to a compatible dtype first.
  rel_suzaku.loc[type_recog, 'target_category'] = rel_suzaku.loc[type_recog, 'target_category'].apply(


,ra,dec,ObsID,science_usable,start,end,duration,target_category,xis0_expo,xis0_num_modes,xis1_expo,xis1_num_modes,xis2_expo,xis2_num_modes,xis3_expo,xis3_num_modes
0,91.1523,-86.6779,701018010,True,2006-04-13 16:24:07.999998,2006-04-14 01:52:19.000001,0 days 09:28:11.000003,EGS,0 days 05:29:41.700000,2,0 days 05:29:41.700000,2,0 days 05:29:25.700000,2,0 days 05:29:33.700000,2
1,48.9864,-85.5003,404019010,True,2009-07-16 14:26:31.000001,2009-07-18 03:30:15.999998,1 days 13:03:44.999997,GS,0 days 17:31:24.300000,2,0 days 17:31:24.300000,2,0 days 00:00:00,0,0 days 17:31:24.300000,2
2,239.2836,-79.2302,703059010,True,2008-10-13 15:35:23.999997,2008-10-15 21:37:21.000000,2 days 06:01:57.000003,EGS,0 days 22:49:57,2,0 days 22:49:43.500000,2,0 days 00:00:00,0,0 days 22:49:35.500000,2
3,265.9438,-76.3446,705013010,True,2010-04-14 00:16:05.000002,2010-04-14 16:45:11.000002,0 days 16:29:06,EGS,0 days 11:46:23.600000,2,0 days 11:46:18.300000,2,0 days 00:00:00,0,0 days 11:46:29,2
4,74.6207,-75.2810,404036010,True,2009-06-14 01:51:18.000003,2009-06-16 13:27:19.000002,2 days 11:36:00.999999,GS,1 days 05:42:37.800000,2,1 days 05:42:37.800000,2,0 days 00:00:00,0,1 days 05:42:29.800000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3036,247.6537,82.9256,706004010,True,2011-04-16 00:38:47.000003,2011-04-16 10:23:15.000000,0 days 09:44:27.999997,EGS,0 days 05:13:55.700000,3,0 days 05:13:42,2,0 days 00:00:00,0,0 days 05:13:55.600000,2
3037,116.7261,85.6829,804031010,True,2010-02-08 09:53:43.999999,2010-02-08 18:15:19.000002,0 days 08:21:35.000003,GCL,0 days 05:19:56,1,0 days 05:19:56,1,0 days 00:00:00,0,0 days 05:19:56,1
3038,112.8158,85.6965,804030010,True,2010-02-08 18:18:12.000001,2010-02-09 12:00:25.000004,0 days 17:42:13.000003,GCL,0 days 10:17:39.100000,2,0 days 10:17:39.100000,2,0 days 00:00:00,0,0 days 10:17:39.100000,2
3039,112.8340,85.6974,804030020,True,2010-02-10 04:18:27.999997,2010-02-11 06:00:24.000002,1 days 01:41:56.000005,GCL,0 days 14:36:01.100000,2,0 days 14:36:01.100000,2,0 days 00:00:00,0,0 days 14:36:01.100000,2


Quite a few observations are close to members of our sample:

In [27]:
sz.filter_on_positions(samp[['ra', 'dec']].values, search_distances)
sz.filtered_obs_info

,ra,dec,ObsID,science_usable,start,end,duration,target_category,xis0_expo,xis0_num_modes,xis1_expo,xis1_num_modes,xis2_expo,xis2_num_modes,xis3_expo,xis3_num_modes
244,359.7661,-60.9080,801016010,True,2006-12-06 17:08:16.000002,2006-12-09 05:11:24.000000,2 days 12:03:07.999998,GCL,1 days 04:00:45.200000,2,1 days 04:00:37.200000,2,0 days 00:00:00,0,1 days 04:00:53.200000,2
324,303.4605,-57.0338,805036010,True,2010-04-12 04:00:58.999997,2010-04-13 06:31:09.999998,1 days 02:30:11.000001,GCL,0 days 18:24:55.200000,2,0 days 18:24:47.200000,2,0 days 00:00:00,0,0 days 18:24:47.200000,2
327,303.1410,-56.7974,801096010,True,2006-05-06 17:40:17.000003,2006-05-07 09:13:24.000004,0 days 15:33:07.000001,GCL,0 days 05:48:08.500000,1,0 days 05:48:08.500000,1,0 days 05:48:16.500000,1,0 days 05:48:00.500000,1
328,302.8690,-56.6666,801095020,True,2006-10-30 19:58:53.000003,2006-10-31 00:15:23.999996,0 days 04:16:30.999993,GCL,0 days 03:19:39.400000,1,0 days 03:19:39.400000,1,0 days 03:19:23.400000,1,0 days 03:19:31.400000,1
329,302.8177,-56.5600,801095010,True,2006-05-06 07:03:13.999997,2006-05-06 17:39:23.999996,0 days 10:36:09.999999,GCL,0 days 04:48:02.600000,1,0 days 04:47:54.600000,1,0 days 04:47:46.600000,1,0 days 04:47:38.600000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2912,258.2484,64.1467,804041010,True,2010-02-07 07:14:08.000002,2010-02-08 09:44:19.000003,1 days 02:30:11.000001,GCL,0 days 12:22:13.700000,1,0 days 12:22:21.700000,1,0 days 00:00:00,0,0 days 12:22:21.700000,1
2915,258.3088,64.2709,809121010,True,2014-06-02 03:08:50.999997,2014-06-04 09:57:14.999996,2 days 06:48:23.999999,GCL,1 days 03:56:44.500000,2,1 days 04:00:12.500000,2,0 days 00:00:00,0,1 days 04:00:04.500000,2
2917,258.0206,64.3236,808039010,True,2013-11-17 00:22:36.999998,2013-11-17 22:43:19.000001,0 days 22:20:42.000003,GCL,0 days 12:09:28.800000,2,0 days 12:09:28.800000,2,0 days 00:00:00,0,0 days 12:09:28.800000,2
3022,256.0138,78.7112,801061010,True,2006-11-10 20:32:38.999996,2006-11-13 02:00:11.999998,2 days 05:27:33.000002,GCL,1 days 02:31:51.100000,2,1 days 02:31:51.100000,2,0 days 00:00:00,0,1 days 02:31:51.100000,2


### ASCA

ASCA is an older telescope, but still may be able to provide useful information, so we search for relevant observations:

In [28]:
asca = ASCA()
asca.all_obs_info

/mnt/ufs18/home-218/turne540/code/DAXA/daxa/mission/asca.py:125: UserWarning: 5 of the 3079 observations located for ASCA have been removed due to all instrument exposures being zero.
  self._fetch_obs_info()


,ra,dec,ObsID,science_usable,start,end,duration,target_category,sis_exposure,gis_exposure
0,288.0116,5.0542,48002000,True,2000-03-23 05:52:13.440,2000-04-04 00:00:25.920,11 days 18:08:12.480000,XRB,5 days 07:25:04,5 days 14:47:28
1,253.5540,39.8158,78002000,True,2000-03-01 12:08:12.480,2000-03-11 00:00:51.840,9 days 11:52:39.360000,AGN,4 days 00:45:04,5 days 04:32:16
2,14.3115,-22.4431,77036000,True,1999-12-03 11:50:38.400,1999-12-15 19:00:28.800,12 days 07:09:50.400000,AGN,4 days 08:14:08,5 days 00:42:40
3,203.8856,-34.3378,77003000,True,1999-07-19 00:40:19.200,1999-07-29 15:40:27.840,10 days 15:00:08.640000,AGN,4 days 02:03:12,4 days 16:33:20
4,266.4864,-28.9429,48004000,True,2000-03-11 00:08:29.760,2000-03-22 00:00:43.200,10 days 23:52:13.440000,XRB,3 days 22:37:04,4 days 07:30:24
...,...,...,...,...,...,...,...,...,...,...
3069,87.7568,-32.2321,77010010,True,1999-10-11 13:47:08.160,1999-10-11 14:19:32.160,0 days 00:32:24,AGN,0 days 00:06:40,0 days 00:06:40
3070,150.6550,-58.6295,15000090,True,1993-08-05 18:31:58.080,1993-08-05 19:12:43.200,0 days 00:40:45.120000,GS,0 days 00:06:56,0 days 00:05:52
3071,335.1555,-24.7349,66013000,True,1998-11-16 16:24:48.960,1998-11-18 17:34:39.360,2 days 01:09:50.400000,NGS,0 days 14:38:24,0 days 00:00:00
3072,333.8738,-17.8223,77002000,True,1999-11-17 04:45:41.760,1999-11-19 01:35:54.240,1 days 20:50:12.480000,AGN,0 days 16:30:24,0 days 00:00:00


In [29]:
asca.filter_on_positions(samp[['ra', 'dec']].values, search_distances)
asca.filtered_obs_info

/mnt/ufs18/home-218/turne540/code/DAXA/daxa/mission/base.py:108: UserWarning: The mission has FoVs defined for SIS0, SIS1, GIS2, GIS3, but only one search_radius has been supplied. You may wish to pass a dictionary of search radii.
  any_ret = change_func(*args, **kwargs)


,ra,dec,ObsID,science_usable,start,end,duration,target_category,sis_exposure,gis_exposure
189,202.0721,-31.4448,87042000,True,2000-01-21 18:24:28.800,2000-01-24 18:10:56.640,2 days 23:46:27.840000,GCL,0 days 20:34:40,0 days 20:47:28
224,359.7480,-60.6878,86010010,True,1998-11-19 15:13:14.880,1998-11-21 13:00:54.720,1 days 21:47:39.840000,GCL,0 days 13:39:44,0 days 18:41:04
225,194.7460,-1.7036,84021000,True,1996-01-10 14:23:34.080,1996-01-12 07:50:44.160,1 days 17:27:10.080000,GCL,0 days 17:20:00,0 days 18:27:44
276,356.9950,-28.0256,83004000,True,1995-06-06 22:06:23.040,1995-06-08 07:35:28.320,1 days 09:29:05.280000,GCL,0 days 13:50:24,0 days 16:35:44
380,258.3540,64.0508,84012000,True,1997-02-21 01:29:08.160,1997-02-22 04:45:41.760,1 days 03:16:33.600000,GCL,0 days 12:39:28,0 days 14:22:56
...,...,...,...,...,...,...,...,...,...,...
2288,227.4190,5.9930,83040020,True,1995-02-02 21:23:45.600,1995-02-03 03:45:21.600,0 days 06:21:36,GCL,0 days 03:02:56,0 days 03:32:16
2353,227.8900,6.3537,83040000,True,1995-01-30 18:14:41.280,1995-01-31 00:40:53.760,0 days 06:26:12.480000,GCL,0 days 03:14:08,0 days 03:25:20
2374,227.8500,5.9012,83040010,True,1995-01-31 00:41:28.320,1995-01-31 09:45:56.160,0 days 09:04:27.840000,GCL,0 days 03:26:56,0 days 03:22:08
2386,255.8760,78.8168,10004020,True,1993-04-09 04:12:00.000,1993-04-09 16:30:34.560,0 days 12:18:34.560000,GS,0 days 02:37:52,0 days 03:21:20


## Setting up a DAXA Archive

We can now create a DAXA archive using the filtered missions, it will automatically download the relevant data. This Archive is responsible for versioning the data and will allow us to make updates in the future, recording all changes made:

In [8]:
if 'LoVoCCS' not in os.listdir('../data/archives'):
    miss_list = [xm, ch, ra, rp, nu, er, ea, sz, asca, sw]
else:
    miss_list = None

arch = Archive('LoVoCCS', miss_list)

/mnt/ufs18/home-218/turne540/code/DAXA/daxa/mission/xmm.py:83: UserWarning: 140 of the 17976 observations located for this mission have been removed due to NaN RA or Dec values
  self._fetch_obs_info()
/mnt/ufs18/home-218/turne540/code/DAXA/daxa/archive/base.py:280: UserWarning: Proprietary data have been selected, but no credentials provided; as such the proprietary data have been excluded from download and further processing.
  cur_miss.download()
/mnt/ufs18/home-218/turne540/code/DAXA/daxa/archive/base.py:280: UserWarning: The number of cores assigned to XMMPointed downloads has been capped at 10, this will minimise dropped connections.
  cur_miss.download()
/mnt/ufs18/home-218/turne540/code/DAXA/daxa/archive/base.py:280: UserWarning: The raw data for this mission have already been downloaded.
  cur_miss.download()
/mnt/ufs18/home-218/turne540/code/DAXA/daxa/archive/base.py:280: UserWarning: The raw data for this mission have already been downloaded.
  cur_miss.download()
/mnt/ufs18

We can use the archive's `info()` method to give us an overall summary of the amount of data available for our LoVoCCS clusters - though of course we have not yet included every applicable X-ray telescope in DAXA, so further pertinent data may yet be available.

In [9]:
arch.info()


-----------------------------------------------------
Version - 0.0.0
Number of missions - 10
Total number of observations - 2734
Beginning of earliest observation - 1990-06-17 17:44:17.000005
End of latest observation - 2024-09-01 23:01:01.999998

-- XMM-Newton Pointed --
   Internal DAXA name - xmm_pointed
   Chosen instruments - PN, M1, M2, R1, R2
   Number of observations - 404
   Fully Processed - False

-- Chandra --
   Internal DAXA name - chandra
   Chosen instruments - ACIS-I, ACIS-S, HRC-I, HRC-S
   Number of observations - 657
   Fully Processed - False

-- RASS --
   Internal DAXA name - rosat_all_sky
   Chosen instruments - PSPC
   Number of observations - 149
   Fully Processed - False

-- ROSAT Pointed --
   Internal DAXA name - rosat_pointed
   Chosen instruments - PSPCB, PSPCC, HRI
   Number of observations - 224
   Fully Processed - False

-- NuSTAR Pointed --
   Internal DAXA name - nustar_pointed
   Chosen instruments - FPMA, FPMB
   Number of observations - 35
   

## Processing the X-ray data

Here we run DAXA processing for the X-ray data we have acquired. **Currently this only processes XMM data, as that is the mission we are initially interested in**.

_**The total storage required for our processed data, and the raw/pre-processed data downloaded for our X-ray missions is currently ~420GB.**_

### XMM-Newton

We use the standard settings for DAXA processing of XMM data - this creates cleaned, combined, event lists for PN, MOS1, and MOS2, along with images and exposure maps. The backend processing makes use of standard SAS and eSAS tools. It also assembles and cleans RGS data, without assembling any further products as that requires knowledge of the region of interest:

In [33]:
full_process_xmm(arch)

XMM-Newton Pointed - Generating calibration files: 100%|██████████| 404/404 [1:26:06<00:00, 12.79s/it]  
XMM-Newton Pointed - Generating ODF summary files: 100%|██████████| 404/404 [07:46<00:00,  1.15s/it]
XMM-Newton Pointed - Assembling RGS event lists: 100%|██████████| 766/766 [02:23<00:00,  5.34it/s]
XMM-Newton Pointed - Correcting RGS for aspect drift: 100%|██████████| 766/766 [00:50<00:00, 15.09it/s]
XMM-Newton Pointed - Cleaning RGS event lists: 100%|██████████| 766/766 [00:44<00:00, 17.09it/s]
XMM-Newton Pointed - Assembling PN and PN-OOT event lists: 100%|██████████| 378/378 [2:17:51<00:00, 21.88s/it]  
XMM-Newton Pointed - Assembling MOS event lists: 100%|██████████| 795/795 [29:15<00:00,  2.21s/it] 
XMM-Newton Pointed - Finding PN/MOS soft-proton flares: 100%|██████████| 1170/1170 [29:09<00:00,  1.50s/it] 
XMM-Newton Pointed - Generating cleaned PN/MOS event lists: 100%|██████████| 1147/1147 [02:12<00:00,  8.64it/s]
XMM-Newton Pointed - Generating final PN/MOS event lists: 10

### All other missions

For now, we use the `preprocessed_in_archive` function to organise all the other mission's data into the archive structure - in the future some may be reprocessed using DAXA - but as we are mainly focused on XMM that is not necessary straight away:

In [10]:
preprocessed_in_archive(arch)

Including pre-processed Chandra data in the archive: 100%|██████████| 657/657 [00:32<00:00, 19.94it/s]
Including pre-processed RASS data in the archive: 100%|██████████| 149/149 [00:30<00:00,  4.91it/s]
Including pre-processed ROSAT Pointed data in the archive: 100%|██████████| 224/224 [00:38<00:00,  5.85it/s]
Including pre-processed NuSTAR Pointed data in the archive: 100%|██████████| 35/35 [00:12<00:00,  2.76it/s]
Including pre-processed eROSITACalPV data in the archive: 100%|██████████| 6/6 [00:05<00:00,  1.19it/s]
Including pre-processed eRASS DE:1 data in the archive: 100%|██████████| 93/93 [06:21<00:00,  4.10s/it]
Including pre-processed Suzaku data in the archive: 100%|██████████| 129/129 [00:10<00:00, 12.10it/s]
Including pre-processed ASCA data in the archive: 100%|██████████| 81/81 [00:01<00:00, 60.46it/s]
Including pre-processed Swift data in the archive:   0%|          | 0/956 [00:00<?, ?it/s]


PreProcessedNotSupportedError: This mission (Swift) does not support the download of pre-processed event lists for the UVOT instrument, so a path cannot be provided. 

## Adding source regions to the Archive

Source regions are a key part of any analysis, but DAXA does not currently have the capability to apply source detection to observations that it processes. As such we shall use source regions generated by the XMM Cluster Survey's (XCS) XAPA source finder. Passing them into the DAXA archive will store them in the DAXA archive directory structure:

In [36]:
reg_path = "../sample_files/xcs_xmm_regions/{o_id}_sky_regions.reg"

In [37]:
success_oi = [oi for oi, success in arch.final_process_success['xmm_pointed'].items() if success 
              and os.path.exists(reg_path.format(o_id=oi))]

reg_dec = {'xmm_pointed': {o: {'regions': reg_path.format(o_id=o)} for o in success_oi}}
arch.source_regions = reg_dec

We find that there are some missing region files for ObsIDs that were successfully processed - we will need to address this:

In [38]:
for g_oi in [oi for oi, success in arch.final_process_success['xmm_pointed'].items() if success]:
    if g_oi not in success_oi:
        print(g_oi)

0201902101
0400010201
0720250401
0761550101
0761210101
0800380101
0843890101
0844050601
0843890201
0881210101
0903510101
0901830601
0904610101
0904610201
0904610401
0904610301
0921880401
0921880801
0920740101
0763940535
0763940536
